In [2]:
from google.colab import files
files.upload()

Saving All_good_data(lemmatization).csv to All_good_data(lemmatization).csv


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('All_good_data(lemmatization).csv')

In [5]:
df['HypРекомендация'] = 0
df['HypЗапрос'] = 0
df['HypПланировать'] = 0
df['HypФото'] = 0
df['HypГод'] = 0
df['HypРемонт'] = 0
df['HypРабота'] = 0
df['HypРассмотреть'] = 0
df['HypВыезд'] = 0

In [ ]:
for i in range(0, len(df)):

  t = []
  for word in str.split(df['responsibleperson_msg'][i],' '):
    if (len(word) > 2 ) and (word[2] != '\''):

        start = 0
        end = -1

        while (word[start] == '\'' or word[start] == '['):
          start += 1
        while (word[end - 1] == '\'' or word[end - 1] == ']'):
          end -= 1

        t.append(word[start:end])

    if (word[2] == '\'') and (len(word) > 4):

      for more_words in str.split(word, ' '):

        start = 0
        end = -1

        while (more_words[start] == '\'' or more_words[start] == '['):
          start += 1
        while (more_words[end - 1] == '\'' or more_words[end - 1] == ']'):
          end -= 1

        t.append(more_words[start:end])

  #start hypothesises
  df['HypФото'][i] = int(df['is_photo_answer'][i])
  
  for word in t:

    if word == 'выезд':
      df['HypВыезд'][i] = 1

    if word == 'год':
      df['HypГод'][i] = 1

    if (word == 'запрос'):
      df['HypЗапрос'][i] = 1

    if (word == 'рассмотреть') or ( word == 'рассмотрение'):
      df['HypРассмотреть'][i] = 1

    if (word == 'план') or (word == 'планировать') or (word == 'запланировать') or (word == 'планироваться') or (word == 'плановый'):
      df['HypПланировать'][i] = 1

    if (word == 'рекомендация'):
      df['HypРекомендация'][i] = 1

    if (word == 'работа'):
      df['HypРабота'][i] = 1

    if (word == 'ремонт'):
      df['HypРемонт'][i] = 1

In [7]:
del df['Unnamed: 0']
del df['Unnamed: 0.1']

In [8]:
values = {'moderateperson_msg': 'NS', 'privateperson_msg': 'NS', 'moderateperson_name': 'NS', 'days_to_solve': df['days_to_solve'].mode()[0]}
df  = df.fillna(values)

In [9]:
!pip install navec

In [10]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2020-11-17 09:02:33--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  15.4MB/s    in 3.3s    

2020-11-17 09:02:38 (15.4 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [11]:
from navec import Navec

path = './navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [12]:
to_preprocess = list(['description','name', 'executor', 'responsibleperson_msg',	'moderateperson_msg',	'privateperson_msg'	,'responsibleperson_name'	,'moderateperson_name'])

In [13]:
vector = [[0 for column in to_preprocess] for i in range(0, len(df))]

for column in range(0, len(to_preprocess)):
  for i in range(0, len(df)):

    t = []
    for word in str.split(df[to_preprocess[column]][i],' '):
      if (len(word) > 2 ) and (word[2] != '\''):
        if (word[2:-2] in navec):
          t.append(navec[word[2:-2]])
    vector[i][column] = t

In [14]:
concat_embed = [[[] for column in to_preprocess] for i in range(0, len(df))]

for j in range(0, len(to_preprocess)):

  m_len = 0
  for i in range(0, len(df)):
    
    concat_embed[i][j] = np.zeros(300)
    if (len(vector[i][j]) > 0):
      for k in range(0, len(vector[i][j])):
        concat_embed[i][j] = concat_embed[i][j] +  vector[i][j][k]

In [15]:
embeddings = pd.DataFrame(concat_embed, columns=to_preprocess)

In [16]:
embeddings = embeddings.merge(df[{'is_photo_request', 'is_photo_answer', 'days_to_solve', 'status', 'HypРекомендация','HypЗапрос', 'HypПланировать', 'HypФото', 'HypГод', 'HypРемонт', 'HypРабота', 'HypРассмотреть','HypВыезд' }], how = 'right', left_index=True, right_index=True)

In [17]:
embeddings['target'] = 1

In [44]:
for i in range(0, len(embeddings)):
  if (embeddings['status'][i] == 'revise'):
    embeddings['target'][i] = 0

3139
4173
5212
5366
5495
5629
5815
6489
6503
6611
6903
7012
7243
7367
7581
7722
8009
8298
8447
8616
8823
9211
9323
9450
9539
9721
10037
10087
10095
10139
10167
10239
10254
10281


In [19]:
features = ['description', 'name', 'executor', 'responsibleperson_msg',
       'moderateperson_msg', 'privateperson_msg', 'responsibleperson_name',
       'moderateperson_name', 'is_photo_request', 'is_photo_answer', 'days_to_solve', 'HypРекомендация','HypЗапрос', 'HypПланировать', 'HypФото', 'HypГод', 'HypРемонт', 'HypРабота', 'HypРассмотреть','HypВыезд']
target = 'target'

In [33]:
X = pd.DataFrame()
for j in range(0, len(features) - 12):
    for k in range(0, 300):
      X[features[j] + '_' + str(k)] =0

In [48]:
Y = embeddings[target]

In [37]:
for i in range(0, len(embeddings)):
  print(i)
  row = {features[j] + '_' + str(k) : embeddings[features[j]][i][k] for k in range(0, 300) for j in range(0, len(features) - 12)}
  row['days_to_solve'] = df['days_to_solve'][i]
  row['is_photo_request'] = df['is_photo_request'][i]
  row['is_photo_answer'] = df['is_photo_answer'][i]
  row['HypРекомендация'] = df['HypРекомендация'][i]
  row['HypЗапрос'] = df['HypЗапрос'][i]
  row['HypПланировать'] = df['HypПланировать'][i]
  row['HypФото'] = df['HypФото'][i]
  row['HypГод'] = df['HypГод'][i]
  row['HypРемонт'] = df['HypРемонт'][i]
  row['HypРабота'] = df['HypРабота'][i]
  row['HypРассмотреть'] = df['HypРассмотреть'][i]
  row['HypВыезд'] = df['HypВыезд'][i]
  X = X.append(row, ignore_index= True)

Выходные данные были обрезаны до нескольких последних строк (5000).
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390
5391
5392
5393
5394
5395
5396
5397
5398
5399
5400
5401
5402
5403
5404
5405
5406
5407
5408
5409
5410
5411
5412
5413
5414
5415
5416
5417
5418
5419
5420
5421
5422
5423
5424
5425
5426
5427
5428
5429
5430
5431
5432
5433
5434
5435
5436
5437
5438
5439
5440
5441
5442
5443
5444
5445
5446
5447
5448
5449
5450
5451
5452
5453
5454
5455
5456
5457
5458
5459
5460
5461
5462
5463
5464
5465
5466
5467
5468
5469
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482
5483
5484
5485
5486
5487
5488
5489
5490
5491
5492
5493
5494
5495
5496
5497
5498
5499
5500
5501
5502
5503
5504
5505
5506
5507
5508
5509
5510
5511
5512
5513
5514
5515
5516
5517
55

In [55]:
from sklearn.naive_bayes import GaussianNB

In [88]:
classifier = GaussianNB()

In [57]:
files.upload()

Saving Razmetka_ready.xlsx to Razmetka_ready.xlsx


{'Razmetka_ready.xlsx': b'PK\x03\x04\x14\x00\x00\x00\x08\x00evoQ\x07AMb\x81\x00\x00\x00\xb1\x00\x00\x00\x10\x00\x00\x00docProps/app.xmlM\x8e=\x0b\x021\x10D\xff\xcaq\xbd\xb7A\xc1Bb@\xd0R\xb0\xb2\x0f{\x1b/\x90dC\xb2B~\xbe9\xc1\x8fn\x1eo\x18F\xdf\ng*\xe2\xa9\x0e-\x86T\x8f\xe3"\x92\x0f\x00\x15\x17\x8a\xb6N]\xa7n\x1c\x97h\xa5cy\x00;\xe7\x91\xce\x8c\xcfHI`\xab\xd4\x1e\xa8\t\xa5\x99\xe6M\xfe\x0e\x8eF\x9fr\x0e\x1e\xadxN\xe6\xea\xb1pe\'\xc3\xa5!\x05\r\xffrm\xde\xa9\xd45\xef&\xf5\x96\x1f\xd6\xf0;i^PK\x03\x04\x14\x00\x00\x00\x08\x00evoQW\xb8Z\x0e\xea\x00\x00\x00\xcb\x01\x00\x00\x11\x00\x00\x00docProps/core.xml\xa5\x91\xc1N\xc30\x0c\x86_e\xea\xbdu\xd21\x0eQ\x97\xcb\xd0NCBb\x12\x88[\xe4x[D\xd3F\x89Q\xbb\xb7\xa7-[\x07\x82\x1b\xc7\xf8\xff\xfc\xd9V*\x0c\n\xdbHO\xb1\r\x14\xd9QZ\xf4\xben\x92\xc2\xb0\xceN\xccA\x01$<\x917\xa9\x18\x88f\x08\x0fm\xf4\x86\x87g<B0\xf8n\x8e\x04\xa5\x10\xf7\xe0\x89\x8d5l`\x14\xe6a6f\x17\xa5\xc5Y\x19>b=\t,\x02\xd5\xe4\xa9\xe1\x04\xb2\x90pc\x99\xa2O\x7f6L\xc9L\xf6\xc9\xcdT\xd7uE\x

In [58]:
old_df = pd.read_excel('Razmetka_ready.xlsx')

In [71]:
X_train = old_df[['is_photo_answer']]

In [72]:
X_train['HypРекомендация'] = 0
X_train['HypЗапрос'] = 0
X_train['HypПланировать'] = 0
X_train['HypФото'] = 0
X_train['HypГод'] = 0
X_train['HypРемонт'] = 0
X_train['HypРабота'] = 0
X_train['HypРассмотреть'] = 0
X_train['HypВыезд'] = 0

In [ ]:
for i in range(0, len(X_train)):

  t = []
  for word in str.split(old_df['responsibleperson_msg'][i],' '):
    if (len(word) > 2 ) and (word[2] != '\''):

        start = 0
        end = -1

        while (word[start] == '\'' or word[start] == '['):
          start += 1
        while (word[end - 1] == '\'' or word[end - 1] == ']'):
          end -= 1

        t.append(word[start:end])

    if (word[2] == '\'') and (len(word) > 4):

      for more_words in str.split(word, ' '):

        start = 0
        end = -1

        while (more_words[start] == '\'' or more_words[start] == '['):
          start += 1
        while (more_words[end - 1] == '\'' or more_words[end - 1] == ']'):
          end -= 1

        t.append(more_words[start:end])

  print(t)
  #start hypothesises
  X_train['HypФото'][i] = int(X_train['is_photo_answer'][i])
  
  for word in t:

    if word == 'выезд':
      X_train['HypВыезд'][i] = 1

    if word == 'год':
      X_train['HypГод'][i] = 1

    if (word == 'запрос'):
      X_train['HypЗапрос'][i] = 1

    if (word == 'рассмотреть') or ( word == 'рассмотрение'):
      X_train['HypРассмотреть'][i] = 1

    if (word == 'план') or (word == 'планировать') or (word == 'запланировать') or (word == 'планироваться') or (word == 'плановый'):
      X_train['HypПланировать'][i] = 1

    if (word == 'рекомендация'):
      X_train['HypРекомендация'][i] = 1

    if (word == 'работа'):
      X_train['HypРабота'][i] = 1

    if (word == 'ремонт'):
      X_train['HypРемонт'][i] = 1

In [78]:
old_df['Lie'].value_counts()

0    64
1    23
Name: Lie, dtype: int64

In [74]:
X_train

,is_photo_answer,HypРекомендация,HypЗапрос,HypПланировать,HypФото,HypГод,HypРемонт,HypРабота,HypРассмотреть,HypВыезд
0,0,0,0,0,0,1,1,1,0,0
1,0,0,0,0,0,1,0,1,0,0
2,0,0,0,0,0,1,1,1,0,0
3,1,0,0,0,1,0,0,1,1,0
4,0,0,0,0,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...
82,0,0,0,0,0,0,0,0,1,0
83,1,0,0,0,1,1,0,1,1,0
84,0,0,0,0,0,0,0,0,0,0
85,1,0,0,0,1,0,0,1,1,0


In [89]:
classifier.fit(X_train[0:60], old_df['Lie'][0:60])

GaussianNB(priors=None, var_smoothing=1e-09)

In [90]:
from sklearn.metrics import accuracy_score, f1_score
print('Testing accuracy %s' % accuracy_score(old_df['Lie'][60:-1], classifier.predict(X_train[60:-1])))
print('Testing F1 score: {}'.format(f1_score(old_df['Lie'][60:-1], classifier.predict(X_train[60:-1]), average='weighted')))

Testing accuracy 0.7307692307692307
Testing F1 score: 0.7397236909432032


In [92]:
df['Result'] = 0

In [98]:
 df['Result'] = classifier.predict(df[['is_photo_answer', 'HypРекомендация', 'HypЗапрос', 'HypПланировать', 'HypФото', 'HypГод', 'HypРемонт', 'HypРабота', 'HypРассмотреть','HypВыезд']])

In [100]:
df.to_csv('Final.csv')
df.to_excel('Final.xlsx')
files.download('Final.csv')
files.download('Final.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>